In [1]:
import os, sys, pandas as pd, numpy as np, string, json, math
from random import randrange, shuffle
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch()

In [3]:
def search_reports(kw="sales", 
                   es_connector=Elasticsearch(),
                   index="test",
                   field="metric_names"
                  
                  ):
    
    """
    Searches for report names using metric name keyword
    """
    
    
    
    fields_list = []
    
    fields_list.append(field)
    
    #search_term_proc = "*" + kw + "*" #adding wildcard
    
    search_term_proc = kw #adding wildcard
    
    
    
    results = es_connector.search(index=index, 
          body={
              "size":10000,
              "query": 
                {"query_string" : { 
                    "query" : search_term_proc,
                    "fields" : ["metric_names*"],
                    
                }
                }
               })
    return pd.json_normalize(results["hits"]["hits"])

search_reports(kw="volume sales_4", field="metric_names.raw", index="test2")#.sort_values(["_score", "_source.metric_names"], ascending=False)

,_index,_type,_id,_score,_source.metric_names,_source.repname,_source.colname
0,test2,_doc,380,8.193352,volume sales_4,reports_synthetic_121_v1_19feb20.xlsx,qld_50
1,test2,_doc,489,8.193352,volume sales_4,reports_synthetic_124_v1_19feb20.xlsx,gold coast_45
2,test2,_doc,764,8.193352,volume sales_4,reports_synthetic_100_v1_19feb20.xlsx,qld_50
3,test2,_doc,340,5.547790,value sales_4,reports_synthetic_126_v1_19feb20.xlsx,auckland_1
4,test2,_doc,580,5.547790,value sales_4,reports_synthetic_139_v1_19feb20.xlsx,melbourne_31
...,...,...,...,...,...,...,...
210,test2,_doc,852,2.150060,sales(volume)_10,reports_synthetic_90_v1_19feb20.xlsx,melbourne_90
211,test2,_doc,856,2.150060,sales(volume)_9,reports_synthetic_140_v1_19feb20.xlsx,victoria_51
212,test2,_doc,927,2.150060,sales(volume)_8,reports_synthetic_37_v1_19feb20.xlsx,melbourne_58
213,test2,_doc,937,2.150060,sales(volume)_6,reports_synthetic_87_v1_19feb20.xlsx,adelaid_73


In [35]:
def search_reports(kw="sales", 
                   es_connector=Elasticsearch(),
                   index="test",
                   field="metric_names"
                  
                  ):
    
    """
    Searches for report names using metric name keyword
    """
    
    
    
    fields_list = []
    
    fields_list.append(field)
    
    kw = kw.strip()
    
    search_term_proc = "*" + kw + "*" #adding wildcard
    
    #search_term_proc = kw #adding wildcard
    
    st2 = "*"+ (" ".join(kw.split("_"))).strip() +"*"
    
    custom_fname = field + ".custom"
    
    std_fname = field + ".standard"
    
    raw_fname = field + ".raw"
    
        
    results = es_connector.search(index=index, 
                                  body={ 
                                   "size":10000,
                                   "query":{ 
                                      "bool":{ 
                                         "should":[ 
                                            { 
                                               "query_string":{ 
                                                  "query":search_term_proc,
                                                  "fields":[ 
                                                     custom_fname
                                                  ],
                                                  "boost":1
                                               }
                                            },
                                            { 
                                               "query_string":{ 
                                                  "query":kw,
                                                  "fields":[ 
                                                     custom_fname
                                                  ],
                                                  "boost":1,
                                                  "fuzziness":100,
                                                  "fuzzy_max_expansions":1000
                                               }
                                            },
                                            { 
                                               "query_string":{ 
                                                  "query":search_term_proc,
                                                  "fields":[ 
                                                     raw_fname
                                                  ],
                                                  "boost":3
                                               }
                                            },
                                            { 
                                               "query_string":{ 
                                                  "query":search_term_proc,
                                                  "fields":[ 
                                                     std_fname
                                                  ],
                                                  "boost":1.5,
                                                  "fuzziness":10,

                                               }
                                            },
                                            { 
                                               "query_string":{ 
                                                  "query":st2,
                                                  "fields":[ 
                                                     custom_fname
                                                  ],
                                                  "boost":1
                                               }
                                            },
                                            { 
                                               "query_string":{ 
                                                  "query":st2,
                                                  "fields":[ 
                                                     raw_fname
                                                  ],
                                                  "boost":3
                                               }
                                            },
                                            { 
                                               "query_string":{ 
                                                  "query":st2,
                                                  "fields":[ 
                                                     std_fname
                                                  ],
                                                  "boost":1.5,
                                                  "fuzziness":10,

                                               }
                                            }
                                         ]
                                      }
                                   }
                                })
    return pd.json_normalize(results["hits"]["hits"])

search_reports(kw="total calls", field="metric_names", index="test2").sort_values(["_score", "_source.metric_names"], ascending=False)

,_index,_type,_id,_score,_source.metric_names,_source.repname,_source.colname
65,test2,_doc,119,9.231398,total calls_9,reports_synthetic_155_v1_19feb20.xlsx,adelaid_61
68,test2,_doc,156,9.231398,total calls_9,reports_synthetic_111_v1_19feb20.xlsx,qld_41
76,test2,_doc,382,9.231398,total calls_9,reports_synthetic_52_v1_19feb20.xlsx,sydney_83
79,test2,_doc,405,9.231398,total calls_9,reports_synthetic_153_v1_19feb20.xlsx,qld_41
90,test2,_doc,603,9.231398,total calls_9,reports_synthetic_38_v1_19feb20.xlsx,adelaid_61
...,...,...,...,...,...,...,...
507,test2,_doc,1378,2.000000,roaming_calls_1,reports_synthetic_138_v1_19feb20.xlsx,qld_258
575,test2,_doc,161,2.000000,roaming_calls_1,reports_synthetic_178_v1_19feb20.xlsx,auckland_11
587,test2,_doc,305,2.000000,roaming_calls_1,reports_synthetic_190_v1_19feb20.xlsx,sydney_47
534,test2,_doc,1842,2.000000,roaming_calls_0,reports_synthetic_125_v1_19feb20.xlsx,nsw_45


In [33]:
search_reports(kw="total calls", field="metric_names", index="test2").sort_values(["_score", "_source.metric_names"], ascending=False)["_source.metric_names"].nunique()

282

In [4]:
df = search_reports(kw="sales", field="metric_names", index="test2")

df["_source.metric_names"].nunique()

196

In [141]:
pd.options.display.max_rows = 1000

In [6]:
es = Elasticsearch()

In [333]:
es.indices.analyze(index="test2", body={
  "analyzer" : "metric_names_analyzer",
  "text" : "arp_"
})

{'tokens': [{'token': 'arp',
   'start_offset': 0,
   'end_offset': 3,
   'type': 'word',
   'position': 0}]}

In [ ]:
es.indices.get_settings("test2")

In [217]:
es.indices.get_mapping("test2")

{'test2': {'mappings': {'properties': {'colname': {'type': 'text',
     'fields': {'custom': {'type': 'text',
       'analyzer': 'metric_names_analyzer'},
      'raw': {'type': 'keyword'},
      'standard': {'type': 'text', 'analyzer': 'standard'}},
     'analyzer': 'metric_names_analyzer'},
    'metric_names': {'type': 'text',
     'fields': {'custom': {'type': 'text',
       'analyzer': 'metric_names_analyzer'},
      'raw': {'type': 'keyword'},
      'standard': {'type': 'text', 'analyzer': 'standard'}},
     'analyzer': 'metric_names_analyzer'},
    'repname': {'type': 'text',
     'fields': {'custom': {'type': 'text',
       'analyzer': 'metric_names_analyzer'},
      'raw': {'type': 'keyword'},
      'standard': {'type': 'text', 'analyzer': 'standard'}},
     'analyzer': 'metric_names_analyzer'}}}}}

In [ ]:
es.indices.open("test")

In [ ]:
es.indices.put_settings(
    index="test",
    body={
  "settings": {
    "analysis": {
      "analyzer": {
        "metric_names_analyzer": {
          "tokenizer": "underscore_tokenizer"
        }
      },
      "tokenizer": {
        "underscore_tokenizer": {
          "type": "pattern",
          "pattern": "_"
        }
      }
    }
  }
}
)

In [ ]:
{'mappings': {
    'properties': {
        'colname': {'type': 'text',
                    'fields': {'metric_names': {'type': 'text', "analyzer":"metric_names_analyzer"}}},
        'metric_names': {'type': 'text',
                         'fields': {'metric_names': {'type': 'text', "analyzer":"metric_names_analyzer"}}},
        'repname': {'type': 'text',
                    'fields': {'metric_names': {'type': 'text', "analyzer":"metric_names_analyzer"}}}
    }
}
}

In [ ]:
es.indices.put_mapping(
    index="test",
    doc_type="my-type",
    body={
        "properties": {  
            "date": {"type":"date"},
            "word_data": {"type": "text"},
            "word_count": {"type": "integer"}
        }
    }
)

In [ ]:
es.get(index="test2", id=1)

In [ ]:
es.search(index="test", body={"query": 
{"fuzzy" : { "metric_names" : 
{"value": "roaming_calls",
"fuzziness": 20
}}}})

In [ ]:
es.search(index="test", 
          body={"query": 
                {"query_string" : { 
                    "query" : "*sales*",
                    "fields" : ["metric_names"]
                           }
                }
               })

In [ ]:
,"fields":{
                              "raw": { 
                                "type":     "keyword",
                                "analyzer": "standard",
                                "search_analyzer":"standard",
                              }
                   }

In [222]:
index_map_setts = {'mappings': {
    'properties': {
        'colname': {'type': 'text',
                    "fields":{
                              "raw": { 
                                "type":     "keyword"
                              },
                           "standard": { 
                                "type":     "text",
                               "analyzer":"standard",
                               "search_analyzer":"standard"
                              },
                           "custom": { 
                                "type":     "text",
                               "analyzer":"metric_names_analyzer",
                               "search_analyzer":"metric_names_analyzer"
                              }
                   }},
        'metric_names': {'type': 'text',
                         "fields":{
                              "raw": { 
                                "type":     "keyword"
                              },
                           "standard": { 
                                "type":     "text",
                               "analyzer":"standard",
                               "search_analyzer":"standard"
                              },
                            "custom": { 
                                "type":     "text",
                               "analyzer":"metric_names_analyzer",
                               "search_analyzer":"metric_names_analyzer"
                              }
                   }},
        'repname': {'type': 'text',
                    "fields":{
                              "raw": { 
                                "type":     "keyword"
                              },
                           "standard": { 
                                "type":     "text",
                               "analyzer":"standard",
                               "search_analyzer":"standard"
                              },
                           "custom": { 
                                "type":     "text",
                               "analyzer":"metric_names_analyzer",
                               "search_analyzer":"metric_names_analyzer"
                              }
                   }},
    }
},
"settings": {
    "analysis": {
      "analyzer": {
        "metric_names_analyzer": {
          "tokenizer": "underscore_tokenizer"
        }
      },
      "tokenizer": {
        "underscore_tokenizer": {
          "type": "pattern",
          "pattern": "_"
        }
      }
    }
  }
 
}


if not es.indices.exists(index="test2"):
    
    print("done")
    
    es.indices.create(index="test2",
         body=index_map_setts)

done


In [221]:
es.indices.delete("test2")

{'acknowledged': True}

In [ ]:
es.indices.create(index="test2",
         body=index_map_setts)

In [ ]:
es.indices.get_mapping("test2")